In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from scipy.ndimage.measurements import label
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from helper import *
from sklearn.cross_validation import train_test_split

%matplotlib inline

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
##Load the data
vehicle_files_dir = './data/vehicles/'
non_vehicle_files_dir = './data/non-vehicles/'
vehicle_files = extract_files(vehicle_files_dir)
non_vehicle_files = extract_files(non_vehicle_files_dir)

print('Number of vehicle files: {}'.format(len(vehicle_files)))
print('Number of non-vehicle files: {}'.format(len(non_vehicle_files)))

#Push images location to cars and not cars data structure
cars = []
notcars = []

for car in vehicle_files:
    cars.append(car)
for objs in non_vehicle_files:
    notcars.append(objs)

Number of vehicle files: 8792
Number of non-vehicle files: 8968


In [3]:
### Training Parameters

### Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = False # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [490, 700] # Min and max in y to search in slide_window()

classifer_pickle = 'svc_car_classifier.pkl'

In [4]:
# Extract the color and hog features
t = time.time()
print('Feature Extraction Started')
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
print('Feature Extraction Ended')
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract features...')

Feature Extraction Started
Feature Extraction Ended
93.86 Seconds to extract features...


In [6]:
## Stack and Preprocess
t = time.time()
print('Feature Preprocessing Started')

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)
print('Feature Preprocessing Ended')
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract features...')

Feature Preprocessing Started
Feature Preprocessing Ended
4.52 Seconds to extract features...


In [7]:
#The Classifier Trainer

print('Using:',orient,'orientations',pix_per_cell,'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

# Check the training time for the SVC

t=time.time()
print('Starting to train at ', t)

parameters = {'kernel':('linear', 'rbf'), 'C':[0, 10]}
svr = svm.SVC(verbose=1)

print('Starting to do Grid Search for parameters..')
svc = GridSearchCV(svr, parameters)
print('Training the classifier..')
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train the classfier...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample

from sklearn.externals import joblib
_ = joblib.dump(svc, classifer_pickle, compress=9)
print('Model saved to Pickle file')

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8364
Starting to train at  1489281233.304897
Starting to do Grid Search for parameters..
Training the classifier..
4534.78 Seconds to train the classfier...
Test Accuracy of SVC =  0.9969
Model saved to Pickle file


In [ ]:
##Finaly after 1.25 Hours got my pickle created